In [2]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import json, re
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Prerequisite : Enable Data Plane V2 (the steps are shown [here](https://drive.google.com/file/d/1RdfFwPYXrIEurT-8cl-S6dULo3V_gynK/view?usp=drive_link)) so that bandwith policy is activated




In [2]:
# Point this to google drive where all results are stored
# Results can be found from :
%cd /content/drive/My Drive/Colab Notebooks/bcgossip/sim/gpbc/cnsim_plosone/bandwidth

/content/drive/My Drive/Colab Notebooks/bcgossip/sim/gpbc/cnsim_plosone/bandwidth


In [3]:
# Load data for default bandwidth
data = pd.read_csv('test-default-bwidth-10X.csv')

# Remove rows where 'message' ends with "-0" (convergence phase)
data = data[~data['message'].str.endswith("-0")]

# Extract number of nodes from 'message'
data['num_nodes'] = data['message'].str.extract(r'cubaan(\d+)-')[0].astype(int)

# Filter rows for 'initiate' and 'received' events
initiate_data = data[data['event_type'] == 'initiate']
received_data = data[data['event_type'] == 'received']

# Group by 'message' and find max 'received' timestamp and 'initiate' timestamp
max_received_timestamps = received_data.groupby('message')['received_timestamp'].max()
initiate_timestamps = initiate_data.groupby('message')['received_timestamp'].first()

# Calculate propagation time in milliseconds (max_received - initiate)
propagation_times_ms = (max_received_timestamps - initiate_timestamps) / 1e6  # Convert nanoseconds to milliseconds
prop = propagation_times_ms.reset_index(name='propagation_time_ms')

# Extract number of nodes from 'message'
prop['num_nodes'] = prop['message'].str.extract(r'cubaan(\d+)').astype(int)

# Group by 'num_nodes' and calculate statistics (in milliseconds)
result_default = prop.groupby('num_nodes')['propagation_time_ms'].agg([
    ('mean (ms)', 'mean'),
    ('min (ms)', 'min'),
    ('median (ms)', 'median'),
    ('max (ms)', 'max')
])

# Round to 2 decimal places for readability
result_default = result_default.round(2)
result_default = result_default.reset_index()
result_default

,num_nodes,mean (ms),min (ms),median (ms),max (ms)
0,10,89.94,42.69,104.50,142.98
1,50,452.00,221.84,491.02,562.17
2,100,1088.87,921.82,1027.00,1481.82
3,200,2083.65,1772.17,2105.52,2228.85
4,400,3943.65,3739.98,3913.75,4203.96
5,600,6028.04,5750.71,5968.19,6420.70


In [4]:
# Load data for 5M
data = pd.read_csv('test-bwidth-5M-10X.csv')

# Remove rows where 'message' ends with "-0" (convergence phase)
data = data[~data['message'].str.endswith("-0")]

# Extract number of nodes from 'message'
data['num_nodes'] = data['message'].str.extract(r'cubaan(\d+)-')[0].astype(int)

# Filter rows for 'initiate' and 'received' events
initiate_data = data[data['event_type'] == 'initiate']
received_data = data[data['event_type'] == 'received']

# Group by 'message' and find max 'received' timestamp and 'initiate' timestamp
max_received_timestamps = received_data.groupby('message')['received_timestamp'].max()
initiate_timestamps = initiate_data.groupby('message')['received_timestamp'].first()

# Calculate propagation time in milliseconds (max_received - initiate)
propagation_times_ms = (max_received_timestamps - initiate_timestamps) / 1e6  # Convert nanoseconds to milliseconds
prop = propagation_times_ms.reset_index(name='propagation_time_ms')

# Extract number of nodes from 'message'
prop['num_nodes'] = prop['message'].str.extract(r'cubaan(\d+)').astype(int)

# Group by 'num_nodes' and calculate statistics (in milliseconds)
result_5M = prop.groupby('num_nodes')['propagation_time_ms'].agg([
    ('mean (ms)', 'mean'),
    ('min (ms)', 'min'),
    ('median (ms)', 'median'),
    ('max (ms)', 'max')
])

# Round to 2 decimal places for readability
result_5M = result_5M.round(2)
result_5M = result_5M.reset_index()
result_5M

,num_nodes,mean (ms),min (ms),median (ms),max (ms)
0,10,103.99,49.13,121.08,139.66
1,50,532.81,295.17,564.65,635.30
2,100,1165.55,1099.70,1144.16,1263.61
3,200,2372.33,2162.93,2278.15,2826.21
4,400,4700.04,4529.79,4715.75,4852.81
5,600,7206.78,6835.58,7133.63,7777.70


In [5]:
# Load data for 30M
data = pd.read_csv('test-bwidth-30M-10X.csv')

# Remove rows where 'message' ends with "-0" (convergence phase)
data = data[~data['message'].str.endswith("-0")]

# Extract number of nodes from 'message'
data['num_nodes'] = data['message'].str.extract(r'cubaan(\d+)-')[0].astype(int)

# Filter rows for 'initiate' and 'received' events
initiate_data = data[data['event_type'] == 'initiate']
received_data = data[data['event_type'] == 'received']

# Group by 'message' and find max 'received' timestamp and 'initiate' timestamp
max_received_timestamps = received_data.groupby('message')['received_timestamp'].max()
initiate_timestamps = initiate_data.groupby('message')['received_timestamp'].first()

# Calculate propagation time in milliseconds (max_received - initiate)
propagation_times_ms = (max_received_timestamps - initiate_timestamps) / 1e6  # Convert nanoseconds to milliseconds
prop = propagation_times_ms.reset_index(name='propagation_time_ms')

# Extract number of nodes from 'message'
prop['num_nodes'] = prop['message'].str.extract(r'cubaan(\d+)').astype(int)

# Group by 'num_nodes' and calculate statistics (in milliseconds)
result_30M = prop.groupby('num_nodes')['propagation_time_ms'].agg([
    ('mean (ms)', 'mean'),
    ('min (ms)', 'min'),
    ('median (ms)', 'median'),
    ('max (ms)', 'max')
])

# Round to 2 decimal places for readability
result_30M = result_30M.round(2)
result_30M = result_30M.reset_index()
result_30M

,num_nodes,mean (ms),min (ms),median (ms),max (ms)
0,10,94.56,45.22,118.66,132.71
1,50,523.38,482.64,514.71,582.55
2,100,1049.79,594.45,1083.98,1185.00
3,200,2068.31,1156.72,2190.39,2237.76
4,400,4541.87,4237.47,4459.86,5140.45
5,600,6635.33,6165.74,6693.99,6854.36


In [6]:
# First merge result_5M and result_30M
comparison = pd.merge(
    result_5M.rename(columns={'mean (ms)': '5 Mbps'}),
    result_30M.rename(columns={'mean (ms)': '30 Mbps'}),
    on='num_nodes',
    how='outer'
)

# Then merge with result_default
comparison = pd.merge(
    comparison,
    result_default.rename(columns={'mean (ms)': 'Default'}),
    on='num_nodes',
    how='outer'
)

# Keep only 'num_nodes' and mean columns
comparison = comparison[['num_nodes', '5 Mbps', '30 Mbps','Default']]

# Display the final comparison table
# print(comparison)
comparison

,num_nodes,5 Mbps,30 Mbps,Default
0,10,103.99,94.56,89.94
1,50,532.81,523.38,452.00
2,100,1165.55,1049.79,1088.87
3,200,2372.33,2068.31,2083.65
4,400,4700.04,4541.87,3943.65
5,600,7206.78,6635.33,6028.04


In [7]:
# Virtualization
# Rename columns for better readability
comparison.rename(columns={'num_nodes': 'Total Nodes', '5 Mbps': '5 Mbps', '30 Mbps': '30 Mbps',
                           'Default': 'Default'}, inplace=True)

# Create a line graph with Plotly
fig = go.Figure()

# Add line plot for Zonal
fig.add_trace(go.Scatter(x=comparison['Total Nodes'], y=comparison['5 Mbps'],
                         mode='lines', line=dict(color='red', width=2),
                         name='5 Mbps Bandwidth'))

# Add line plot for Regional
fig.add_trace(go.Scatter(x=comparison['Total Nodes'], y=comparison['30 Mbps'],
                         mode='lines', line=dict(color='blue', width=2),
                         name='30 Mbps Bandwidth'))

# Add line plot for Regional
fig.add_trace(go.Scatter(x=comparison['Total Nodes'], y=comparison['Default'],
                         mode='lines', line=dict(color='green', width=2),
                         name='Default'))

# Update layout with aesthetic enhancements
fig.update_layout(
    xaxis_title='Total number of nodes',
    yaxis_title='Propagation Time (milliseconds)',
    showlegend=True,
    legend_title_text='Legend',
    xaxis=dict(
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        font=dict(size=14)
    )
)

# Show the plot
fig.show()